# RIPEMD-320 Implementation

A pure Python implementation of the RIPEMD-320 cryptographic hash function.

## Overview

RIPEMD-320 is a 320-bit cryptographic hash function that extends RIPEMD-160 by doubling the hash state size. It maintains the same algorithmic structure as RIPEMD-160 but uses 10 state variables (5 per computation line) instead of 5, resulting in a 320-bit output that provides enhanced security through increased hash length.

## Features

- **Pure Python Implementation**: No external dependencies required
- **320-bit Output**: Provides stronger security than RIPEMD-160
- **Extended State**: Uses 10 state variables (5 per line)
- **Proven Algorithm**: Based on the well-tested RIPEMD-160 structure
- **Flexible Input**: Handles both string and bytes input
- **Periodic State Exchanges**: Variables swap between lines for enhanced security


## Functions

### `RIPEMD_320_hash(message_str)`
Main hashing function:
- Accepts string or bytes input
- Returns 40-byte (320-bit) hash digest
- Processes message in 512-bit chunks
- Uses two parallel 5-variable computation lines

### `ripemd320_hex(message)`
Convenience wrapper function:
- Returns hash as lowercase hexadecimal string
- 80-character output (40 bytes × 2 hex chars)

### Algorithm Components

#### `preprocess(message_bytes)`
Standard RIPEMD message preprocessing:
- Appends 0x80 bit
- Pads with zeros to proper block boundary  
- Appends original message bit length

#### Round Functions
- `f(j, x, y, z)`: Five different boolean functions (same as RIPEMD-160)
- `K(j)`, `k_dash(j)`: Round constants for both computation lines
- `r(j)`, `r_dash(j)`: Message word selection patterns
- `s(j)`, `s_dash(j)`: Left rotation amounts for each round

#### `rol(x, s)`
32-bit left rotation operation.

## Algorithm Specifications

RIPEMD-320 parameters:
- **Block size**: 512 bits (64 bytes)
- **Output size**: 320 bits (40 bytes) 
- **Rounds**: 80 rounds per line (160 total rounds)
- **State variables**: 10 (5 per computation line)
- **Initial values**: Ten 32-bit constants (h0-h9)
- **Round functions**: 5 different boolean functions
- **State exchanges**: Variables swap between lines at specific rounds

## Extended State Design

### Initial Hash Values
```python
h0 = 0x67452301    h1 = 0xEFCDAB89    h2 = 0x98BADCFE    h3 = 0x10325476    h4 = 0xC3D2E1F0
h5 = 0x76543210    h6 = 0xFEDCBA98    h7 = 0x89ABCDEF    h8 = 0x01234567    h9 = 0x3C2D1E0F
```

### Dual Line Processing
- **Left line**: Uses variables A, B, C, D, E and initial values h0-h4
- **Right line**: Uses variables A', B', C', D', E' and initial values h5-h9  
- **State exchanges**: Variables swap between lines at rounds 15, 31, 47, 63, 79

### Round Function Sequence
Both lines use the same 5 round functions as RIPEMD-160:
1. **Rounds 0-15**: `x ^ y ^ z`
2. **Rounds 16-31**: `(x & y) | (~x & z)`
3. **Rounds 32-47**: `(x | ~y) ^ z` 
4. **Rounds 48-63**: `(x & z) | (y & ~z)`
5. **Rounds 64-79**: `x ^ (y | ~z)`

## Example Outputs

```python
# Test cases with expected outputs
print(ripemd320_hex("RIPEMD 320 is a cryptographic hash function.").upper())
# Output: 87BA3D1BBC81E15F1477FC6571F320E35795E6019410FDACB8486A7F282F755CD7DCCFA3E01FBB0B

print(ripemd320_hex("Cryptographic Hash Functions").upper())  
# Output: 4334A52CC817D836F14E014BFD1B257CA825F9F3FD9DCCF778FC5FB691EA5F2320402E68F161C5D9
```

## Security Benefits

RIPEMD-320 provides enhanced security through:

### Increased Hash Length
- **320-bit output**: Significantly more collision-resistant than shorter hashes
- **Extended state**: 10 state variables vs 5 in RIPEMD-160
- **Larger search space**: Makes brute force attacks computationally infeasible

### Algorithmic Strengths
- **Dual line structure**: Two parallel computation paths increase complexity
- **State mixing**: Regular variable exchanges between computation lines
- **Proven design**: Based on extensively analyzed RIPEMD-160 structure
- **No known vulnerabilities**: No practical attacks discovered

## Applications

RIPEMD-320 is suitable for:
- **High-security applications**: Where 160-bit security is insufficient
- **Long-term archival**: Digital signatures requiring extended validity
- **Compliance requirements**: Systems mandating longer hash functions
- **Research purposes**: Academic and cryptographic research
- **Legacy compatibility**: Extending existing RIPEMD-based systems

## Performance Characteristics

- **Block processing**: Efficient 512-bit block handling
- **Memory usage**: Fixed memory footprint regardless of input size
- **Streaming capable**: Can process arbitrarily large inputs
- **Cross-platform**: Pure Python ensures portability

## Technical Implementation Details

### State Variable Management
The algorithm maintains two sets of 5 state variables each:
- Variables are rotated and mixed within each line
- Periodic exchanges ensure both lines contribute to final output
- Each variable undergoes C rotation by 10 bits in RIPEMD style

### Message Block Processing  
- 512-bit blocks split into 16 32-bit words
- Each word used multiple times with different access patterns
- Left rotation amounts vary by round for diffusion
- Constants differ between computation lines

## Standards Compliance

This implementation:
- Follows the RIPEMD-320 specification precisely
- Uses standard initial values and constants
- Implements correct state exchange timing
- Produces output consistent with reference implementations

## Compatibility Requirements

- **Python 3.6+**: Compatible with modern Python versions
- **No dependencies**: Pure Python implementation
- **Memory efficient**: Suitable for resource-constrained environments  
- **Thread safe**: No global state or side effects

In [1]:
def preprocess(message_bytes):
    if(not isinstance(message_bytes, bytes)):
        raise TypeError("Input must be of type bytes")
    m = bytearray(message_bytes)
    bit_length = len(m) * 8

    # Append 0x80 (10000000 in binary)
    m.append(0x80)

    # Compute number of zero bytes to append
    padding_len = (56 - (len(m) % 64)) % 64
    m.extend([0x00] * padding_len)

    # Append original bit length as a 64-bit little-endian integer
    m.extend(bit_length.to_bytes(8, byteorder='little'))

    return m

In [2]:
def f(j,x,y,z):
    if(j>=0 and j<=15):
        return x^ y ^ z
    elif(j>=16 and j<=31):
        return (x & y) | (~x & z)   
    elif(j>=32 and j<=47):
        return (x | ~y) ^ z
    elif(j>=48 and j<=63):
        return (x & z) | (y & ~z)
    elif(j>=64 and j<=79):
        return x ^ (y | ~z)

In [3]:
def K(j):
    if(j>=0 and j<=15):
        return 0x00000000
    elif(j>=16 and j<=31):
        return 0x5A827999
    elif(j>=32 and j<=47):
        return 0x6ED9EBA1
    elif(j>=48 and j<=63):
        return 0x8F1BBCDC
    elif(j>=64 and j<=79):
        return 0xA953FD4E
def k_dash(j):
    if(j>=0 and j<=15):
        return 0x50A28BE6
    elif(j>=16 and j<=31):
        return 0x5C4DD124
    elif(j>=32 and j<=47):
        return 0x6D703EF3
    elif(j>=48 and j<=63):
        return 0x7A6D76E9
    elif(j>=64 and j<=79):
        return 0x00000000   

In [4]:
def r(j):
    if(j>=0 and j<=15):
        return j
    elif(j>=16 and j<=31):
        return [7, 4, 13, 1, 10, 6, 15, 3, 12, 0, 9, 5, 2, 14, 11, 8][j-16]
    elif(j>=32 and j<=47):
        return [3, 10, 14, 4, 9, 15, 8, 1, 2, 7, 0, 6, 13, 11, 5, 12][j-32]
    elif(j>=48 and j<=63):
        return [1, 9, 11, 10, 0, 8, 12, 4, 13, 3, 7, 15, 14, 5, 6, 2][j-48]
    elif(j>=64 and j<=79):
        return [4, 0, 5, 9, 7, 12, 2, 10, 14, 1, 3, 8, 11, 6, 15 ,13][j-64]
def r_dash(j):
    if(j>=0 and j<=15):
        return [5, 14, 7, 0, 9, 2, 11, 4, 13, 6, 15, 8, 1, 10, 3, 12][j]
    elif(j>=16 and j<=31):
        return [6, 11, 3, 7, 0, 13, 5, 10, 14, 15, 8, 12, 4, 9, 1, 2][j-16]
    elif(j>=32 and j<=47):
        return [15, 5, 1, 3, 7, 14, 6, 9, 11, 8, 12, 2, 10, 0, 4, 13][j-32]
    elif(j>=48 and j<=63):
        return [8, 6, 4, 1, 3, 11, 15, 0, 5, 12, 2, 13, 9, 7, 10, 14][j-48]
    elif(j>=64 and j<=79):
        return [12,15 ,10 ,4 ,1 ,5 ,8 ,7 ,6 ,2 ,13 ,14 ,0 ,3 ,9 ,11][j-64]

In [5]:
def s(j):
    if(j>=0 and j<=15):
        return [11, 14, 15, 12, 5, 8, 7, 9, 11, 13, 14, 15, 6, 7, 9, 8][j]
    elif(j>=16 and j<=31):
        return [7, 6, 8, 13, 11, 9, 7, 15, 7, 12, 15, 9, 11, 7, 13, 12][j-16]
    elif(j>=32 and j<=47):
        return [11, 13, 6, 7, 14, 9, 13, 15, 14, 8, 13, 6, 5, 12, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [11,12 ,14 ,15 ,14 ,15 ,9 ,8 ,9 ,14 ,5 ,6 ,8 ,6 ,5 ,12][j-48]
    elif(j>=64 and j<=79):
        return [9 ,15 ,5 ,11 ,6 ,8 ,13 ,12 ,5 ,12 ,13 ,14 ,11 ,8 ,5 ,6][j-64]
def s_dash(j):
    if(j>=0 and j<=15):
        return [8, 9, 9, 11, 13, 15, 15, 5, 7, 7, 8, 11, 14, 14, 12, 6][j]
    elif(j>=16 and j<=31):
        return [9, 13, 15, 7, 12, 8, 9, 11, 7, 7, 12, 7, 6, 15, 13, 11][j-16]
    elif(j>=32 and j<=47):
        return [9, 7, 15, 11, 8, 6, 6, 14, 12, 13, 5, 14, 13, 13, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [15 ,5 ,8 ,11 ,14 ,14 ,6 ,14 ,6 ,9 ,12 ,9 ,12 ,5 ,15 ,8][j-48]
    elif(j>=64 and j<=79):
        return [8 ,5 ,12 ,9 ,12 ,5 ,14 ,6 ,8 ,13 ,6 ,5 ,15 ,13 ,11 ,11][j-64]    

In [6]:
"""
 h0 = 0x67452301; h1 = 0xEFCDAB89; h2 = 0x98BADCFE; h3 = 0x10325476; h4 = 0xC3D2E1F0;
    h5 = 0x76543210; h6 = 0xFEDCBA98; h7 = 0x89ABCDEF; h8 = 0x01234567; h9 = 0x3C2D1E0F;
"""
h0 = 0x67452301; h1 = 0xEFCDAB89; h2 = 0x98BADCFE; h3 = 0x10325476;
h4 = 0xC3D2E1F0; h5 = 0x76543210; h6 = 0xFEDCBA98; h7 = 0x89ABCDEF; h8 = 0x01234567; h9 = 0x3C2D1E0F; 

In [7]:
def rol(x, s):
    return ((x << s) | (x >> (32 - s))) & 0xFFFFFFFF

In [8]:
def RIPEMD_320_hash(message_str):
    # Initial hash values
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0
    h5 = 0x76543210
    h6 = 0xFEDCBA98
    h7 = 0x89ABCDEF
    h8 = 0x01234567
    h9 = 0x3C2D1E0F
    if isinstance(message_str, str):
        message = message_str.encode("utf-8")
    else:
        message = message_str
    padded_message = preprocess(message)
    
    # Process the message in successive 512-bit chunks
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        
        # Break chunk into sixteen 32-bit little-endian words
        X = []
        for j in range(16):
            X.append(int.from_bytes(block[j*4:j*4+4], byteorder='little'))
        
        # Initialize hash value for this chunk
        A = h0; B = h1; C = h2; D = h3; E = h4
        A_prime = h5; B_prime = h6; C_prime = h7; D_prime = h8; E_prime = h9;
        
        # Main loop
        for j in range(80):
            # Left line
            T = (A + f(j, B, C, D) + X[r(j)] + K(j)) & 0xFFFFFFFF
            T = rol(T, s(j))
            T = (T + E) & 0xFFFFFFFF
            A = E
            E = D
            D = rol(C, 10)
            C = B
            B = T
            
            # Right line
            T_prime = (A_prime + f(79-j, B_prime, C_prime, D_prime) + X[r_dash(j)] + k_dash(j)) & 0xFFFFFFFF
            T_prime = rol(T_prime, s_dash(j))
            T_prime = (T_prime + E_prime) & 0xFFFFFFFF
            A_prime = E_prime
            E_prime = D_prime
            D_prime = rol(C_prime, 10)
            C_prime = B_prime
            B_prime = T_prime
            if(j==15):
                T= B ;  B = B_prime ; B_prime = T
            elif(j==31):
                T = D ; D = D_prime ; D_prime = T
            elif(j==47):
                T = A ; A = A_prime ; A_prime = T
            elif(j==63):
                T = C ; C = C_prime ; C_prime = T
            elif(j==79):
                T = E ; E = E_prime ; E_prime = T    
        # Combine results
        h0 = (h0 + A) & 0xFFFFFFFF
        h1 = (h1 + B) & 0xFFFFFFFF  
        h2 = (h2 + C) & 0xFFFFFFFF
        h3 = (h3 + D) & 0xFFFFFFFF
        h4 = (h4 + E) & 0xFFFFFFFF
        h5 = (h5 + A_prime) & 0xFFFFFFFF
        h6 = (h6 + B_prime) & 0xFFFFFFFF
        h7 = (h7 + C_prime) & 0xFFFFFFFF
        h8 = (h8 + D_prime) & 0xFFFFFFFF
        h9 = (h9 + E_prime) & 0xFFFFFFFF
    
    # Produce the final hash value as a 160-bit number (20 bytes)
    return (h0.to_bytes(4, byteorder='little') + 
            h1.to_bytes(4, byteorder='little') + 
            h2.to_bytes(4, byteorder='little') + 
            h3.to_bytes(4, byteorder='little') + 
            h4.to_bytes(4, byteorder='little') +
            h5.to_bytes(4, byteorder='little') + 
            h6.to_bytes(4, byteorder='little') +
            h7.to_bytes(4, byteorder='little') +
            h8.to_bytes(4, byteorder='little') +
            h9.to_bytes(4, byteorder='little'))


def ripemd320_hex(message):
    """Return RIPEMD-160 hash as hexadecimal string"""
    return RIPEMD_320_hash(message).hex()

In [10]:
print(ripemd320_hex("RIPEMD 320 is a cryptographic hash function.").upper())

87BA3D1BBC81E15F1477FC6571F320E35795E6019410FDACB8486A7F282F755CD7DCCFA3E01FBB0B


In [17]:
print(ripemd320_hex("Cryptographic Hash Functions").upper())

4334A52CC817D836F14E014BFD1B257CA825F9F3FD9DCCF778FC5FB691EA5F2320402E68F161C5D9
